---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import re
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
# Return DataFrame of towns and states from list in following format: 
# DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], columns=["State", "RegionName"]  )
# Follow steps in data cleaning below:

def get_list_of_university_towns():
    #1. For "State", removing characters from "[" to the end.
    #2. For "RegionName", when applicable, removing every character from " (" to the end.
    #3. Depending on how you read the data, you may need to remove newline character '\n'.
    with open('university_towns.txt') as file: 
        data = []
        for line in file:
            data.append(line[:-1])

    state_town = []        
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])

        else:
            town = line
            state_town.append([state,town])  

    univ_towns = pd.DataFrame(state_town,columns=["State", "RegionName"])
    
    return univ_towns

In [6]:
def get_recession_start():
    # Recession start defined as beginning of two quarter period in which both quarters have GDP decline
    # Load the gdp data file, skip header rows
    df = pd.read_excel('gdplev.xls')
    gdp = df.drop(range(7))
    # Select columns for Quarter, GDP chained value in 2009 dollars), in quarterly intervals 
    gdp = gdp[['Unnamed: 4', 'Unnamed: 6']]
    gdp.columns = ['Quarter', 'GDP']
    # Select GDP data from first quarter of 2000 onward.
    gdp = gdp.loc[219:].reset_index(drop=True)
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])

    # compute difference between a value and its predecessor in a series using
    #df['diff'] = df['gdp'].diff()
    quarters = []
    for i in range(len(gdp)-2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) and (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            quarters.append(gdp.iloc[i][0])

    #'''Returns year, quarter of recession start time as string value in a format such as 2005q3
    return(quarters[1])

In [7]:
# Recession starts in 2008q3, find year, quarter at which gdp shows growth

def get_recession_end():
    df = pd.read_excel('gdplev.xls')
    gdp = df.drop(range(7))
    gdp = gdp[['Unnamed: 4', 'Unnamed: 6']]
    gdp.columns = ['Quarter', 'GDP']
    gdp = gdp.loc[219:].reset_index(drop=True)
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])

    # New index from start of recession 
    start = get_recession_start()
    start_index = gdp[gdp['Quarter'] == start].index.tolist()[0]
    gdp = gdp.iloc[start_index:]

    quarters = []
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] < gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] < gdp.iloc[i][1]):
            quarters.append(gdp.iloc[i][0])

    # Returns year, quarter of recession end time as string value
    return(quarters[0])

In [8]:
# Recession occurs between 2008q3 to 2009q4; find quarter in interval with lowest gdp  

def get_recession_bottom():
    df = pd.read_excel('gdplev.xls')
    gdp = df.drop(range(7))
    gdp = gdp[['Unnamed: 4', 'Unnamed: 6']]
    gdp.columns = ['Quarter', 'GDP']
    gdp = gdp.loc[219:].reset_index(drop=True)
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])

    start = get_recession_start()
    start_index = gdp[gdp['Quarter'] == start].index.tolist()[0]
    end = get_recession_end()
    end_index = gdp[gdp['Quarter'] == end].index.tolist()[0]
    gdp = gdp.iloc[start_index: end_index+1]

    bottom = gdp['GDP'].min()
    bottom_index = gdp[gdp['GDP'] == bottom].index.tolist()[0]-start_index
    # Returns year, quarter of recession end time as string value
    return(gdp.iloc[bottom_index]['Quarter'])

In [10]:
# Quarters are specific three month periods: Q1: Jan-March, Q2: Apr-June, Q3: July-Sept    
# Generate new column names

def new_col_names():
    year = list(range(2000,2017))
    quart = ['q1', 'q2', 'q3', 'q4']
    quart_year = []
    for i in year:
        for j in quart:
            quart_year.append((str(i)+j))
    return quart_year[:67]

# Convert housing data to quarters and return mean values in DF

def convert_housing_data_to_quarters():
    # DF with columns for 2000q1 through 2016q3, should have multi-index in shape of ["State","RegionName"].
    homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    homes.drop(['Metro', 'CountyName', 'RegionID', 'SizeRank'], axis=1, inplace=True)
    homes['State'] = homes['State'].map(states)
    homes.set_index(['State', 'RegionName'], inplace=True)
    
    cols = list(homes.columns)
    homes.drop(cols[0:45], axis=1, inplace=True)

    # Quarters of year
    quarters = [list(homes.columns)[x:x+3] for x in range(0, len(list(homes.columns)), 3)]
    col_names = new_col_names()    
    
    for col,q in zip(col_names, quarters):
        homes[col] = homes[q].mean(axis=1)
    
    homes = homes[col_names]
    return(homes)

In [11]:
def run_ttest():
    # Recession start = 2008q3, Recession bottom = 2009q2
    start = get_recession_start()
    bottom = get_recession_bottom()
    data = convert_housing_data_to_quarters().copy()

    # Create new data showing decline or growth of housing prices 
    # between the Recession start and Recession bottom. 
    recession = data.loc[:, '2008q3':'2009q4'] 
    recession = recession.reset_index()
    recession['Difference'] = recession[start] - recession[bottom]

    # Compare university town values to non-university towns values, 
    # Get list of university towns and use set to obtain unique values
    univ_towns = get_list_of_university_towns()['RegionName']
    univ_towns = set(univ_towns)

    # Create column for grouping cities as university towns
    def is_univ_town(row):
        if row['RegionName'] in univ_towns:
            return 1
        else:
            return 0
    recession['UnivTown'] = recession.apply(is_univ_town, axis = 1)

    # Group towns as univ_towns and non_univ towns:
    uni_town = recession[recession['UnivTown']==1].loc[:,"Difference"].dropna()
    non_univ = recession[recession['UnivTown']==0].loc[:,"Difference"].dropna()

    def better():
        if non_univ.mean() < uni_town.mean():
            return 'non-univesity town'
        else:
            return 'university town'

    # Conduct ttest to btain p-value, determine whether null hypothesis can be rejected 
    p_val = list(ttest_ind(non_univ, uni_town))[1]    
    p_val    
    
    if p_val < 0.01:
        different = True
    else:
        different = False    

    result = (different, p_val, better())

    # The variable p should be equal to the exact p value returned from scipy.stats.ttest_ind(). 
    # The value for better should be either "university town" or "non-university town", 
    # depending on which has a lower mean price ratio (which is equivilent to reduced market loss).
    return(result)

